In [2]:
# 공부 출처 : https://hipster4020.tistory.com/113
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
movies = pd.read_csv("C:/Users/82103/data/TMDB_movie/tmdb_5000_movies.csv")
print(movies.shape)
print(movies.columns)
movies.head()


(4803, 20)
Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [9]:
movies.isnull().sum()

budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
dtype: int64

In [19]:
movies=movies[['id', 'title', 'genres', 'vote_average', 'vote_count']]

In [20]:
def extract(obj):
    l = []
    for i in literal_eval(obj): # 문자열 상태인 dict를 dictionary type으로 바꿈
        l.append(i['name'])
    return l

In [22]:
movies['genres'] = movies['genres'].apply(extract)
movies['genres'].head()

In [24]:
movies['genres_literal'] = movies['genres'].apply(lambda x : (' ').join(x))
movies.head()

,id,title,genres,vote_average,vote_count,genres_literal
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",7.2,11800,Action Adventure Fantasy Science Fiction
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]",6.9,4500,Adventure Fantasy Action
2,206647,Spectre,"[Action, Adventure, Crime]",6.3,4466,Action Adventure Crime
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]",7.6,9106,Action Crime Drama Thriller
4,49529,John Carter,"[Action, Adventure, Science Fiction]",6.1,2124,Action Adventure Science Fiction


In [26]:
# min_df는 너무 드물게로 나타나는 용어를 제거하는 데 사용. min_df = 0.01은 "문서의 1 % 미만"에 나타나는 용어를 무시한다. 
# ngram_range는 n-그램 범위.
count_vect = CountVectorizer(min_df = 0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies['genres_literal'])
print(genre_mat.shape)

(4803, 276)


In [27]:
genre_sim=cosine_similarity(genre_mat, genre_mat)

In [38]:
# [:, ::-1] axis = 1 기준으로 2차원 numpy 배열 뒤집기
# argsort란 오름차순으로 정렬하면서 인덱스로 표현 => 그러므로 뒤집기
genre_sim_sorted_ind=genre_sim.argsort()[:, ::-1] 
print(genre_sim_sorted_ind[1,:20])

[ 262    1  129  199  206  208   98 2343 2390  379  329   12  330  332
 1266   50  784  215  786 2655]


In [29]:
movies[['title', 'vote_average', 'vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


In [30]:
percentile = 0.6
m = movies_df['vote_count'].quantile(percentile)  # 평점을 부여하기 위한 최소 평가 수
C = movies_df['vote_average'].mean()  # 전체 영화의 평균 평점

def weighted_vote_average(record):
  v = record['vote_count']  # 영화에 평가를 매긴 횟수
  R = record['vote_average']  # 영화의 평균 평점

  return ( (v/(v+m)) * R ) + ( (m/(m+v)) * C )  # 가중 평점 계산 식

movies_df['weighted_vote'] = movies.apply(weighted_vote_average, axis=1)

In [31]:
movies_df[['title', 'weighted_vote', 'vote_count']].sort_values('weighted_vote', ascending=False)[:10]

,title,weighted_vote,vote_count
1881,The Shawshank Redemption,8.396052,8205
3337,The Godfather,8.263591,5893
662,Fight Club,8.216455,9413
3232,Pulp Fiction,8.207102,8428
65,The Dark Knight,8.136930,12002
1818,Schindler's List,8.126069,4329
3865,Whiplash,8.123248,4254
809,Forrest Gump,8.105954,7927
2294,Spirited Away,8.105867,3840
2731,The Godfather: Part II,8.079586,3338


In [39]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
  title_movie = df[df['title'] == title_name]
  title_index = title_movie.index.values

  # top_n의 2배에 해당하는 장르 유사성이 높은 인덱스 추출
  similar_indexes = sorted_ind[title_index, :(top_n*2)]
  # reshape(-1) 1차열 배열 반환
  similar_indexes = similar_indexes.reshape(-1)
  # 기준 영화 인덱스는 제외
  similar_indexes = similar_indexes[similar_indexes != title_index]

  # top_n의 2배에 해당하는 후보군에서 weighted_vote가 높은 순으로 top_n만큼 추출
  return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

similar_movies=find_sim_movie(movies_df, genre_sim_sorted_ind, 'No Country for Old Men', 10)
similar_movies[['title', 'vote_count', 'weighted_vote']]

,title,vote_count,weighted_vote
3234,Nightcrawler,3395,7.451748
1394,Donnie Brasco,1147,7.080887
867,The Godfather: Part III,1546,6.905293
1105,Sleepers,715,6.887967
1328,The Town,1468,6.817170
4226,Nine Queens,153,6.474621
1038,The Infiltrator,573,6.400681
2240,A Most Violent Year,546,6.335213
3789,Trust,307,6.322389
3091,Killer Joe,425,6.310138


TypeError: list indices must be integers or slices, not tuple